# Test Open Data Zürich

Dieses Notebook verwendet als Vorlage die Webseite «[Kitas in Berlin - Jupyter Notebook](https://juanitorduz.github.io/kitas_berlin/)»

Mit Colab kann **dieses Jupyter-Notebook interaktiv im Browser** gestartet werden:
[![Open In Colab](https://colab.research.google.com/assets/colab-badge.svg)](https://colab.research.google.com/github/DonGoginho/myPy_RIS_API/blob/main/RIS_API_AL_Geschaefte.ipynb)


**Inhaltsverzeichnis:**

1. [Daten importieren](#Mitglieder-der-AL)
2. [Daten prüfen](#Gemeinderats-Geschäfte)
3. [Daten analysieren](#Gemeinderats-Geschäfte)



## Importiere notwendige Packages:

In [4]:
import numpy as np
import pandas as pd
import geopandas as gpd
import mplleaflet
import matplotlib.pyplot as plt
import matplotlib.ticker as mtick
import seaborn as sns
sns.set_style(
    style='darkgrid', 
    rc={'axes.facecolor': '.9', 'grid.color': '.8'}
)
sns.set_palette(palette='deep')
sns_c = sns.color_palette(palette='deep')
%matplotlib inline
from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()

plt.rcParams['figure.figsize'] = [10, 6]
plt.rcParams['figure.dpi'] = 100

pd.options.display.float_format = '{:.0f}'.format
#pd.set_option('display.width', 100)
#pd.set_option('display.max_columns', 15)

## Lies Daten ein


In [5]:
"""
Versuch:
Import Daten via Datastore

Missing:
Korrektes Einlesen des JSON-Outputs

------------
import requests
SSL_VERIFY = True
# evtl. SSL_VERIFY auf False setzen wenn die Verbindung zu https://www.gemeinderat-zuerich.ch nicht klappt (z.B. wegen Proxy)
# Um die SSL Verifikation auszustellen, bitte die nächste Zeile einkommentieren ("#" entfernen)
# SSL_VERIFY = False

---------------
if not SSL_VERIFY:
    import urllib3
    urllib3.disable_warnings()
-------------    

-----------------------------------------------------
#https://data.stadt-zuerich.ch/api/3/action/datastore_search?resource_id=a90b0184-c809-48b4-87ca-91e5811da885

query = {'resource_id': 'a90b0184-c809-48b4-87ca-91e5811da885'} #, 'records':'StichtagDatJahr' }
headers = {'Accept': 'application/json'}
r = requests.get('https://data.stadt-zuerich.ch/api/3/action/datastore_search', 
                 params=query, 
                 headers=headers, 
                 verify=SSL_VERIFY)
data = r.json()
print('done')
#data
-----------------------------------------------------
"""

'\nVersuch:\nImport Daten via Datastore\n\nMissing:\nKorrektes Einlesen des JSON-Outputs\n\n------------\nimport requests\nSSL_VERIFY = True\n# evtl. SSL_VERIFY auf False setzen wenn die Verbindung zu https://www.gemeinderat-zuerich.ch nicht klappt (z.B. wegen Proxy)\n# Um die SSL Verifikation auszustellen, bitte die nächste Zeile einkommentieren ("#" entfernen)\n# SSL_VERIFY = False\n\n---------------\nif not SSL_VERIFY:\n    import urllib3\n    urllib3.disable_warnings()\n-------------    \n\n-----------------------------------------------------\n#https://data.stadt-zuerich.ch/api/3/action/datastore_search?resource_id=a90b0184-c809-48b4-87ca-91e5811da885\n\nquery = {\'resource_id\': \'a90b0184-c809-48b4-87ca-91e5811da885\'} #, \'records\':\'StichtagDatJahr\' }\nheaders = {\'Accept\': \'application/json\'}\nr = requests.get(\'https://data.stadt-zuerich.ch/api/3/action/datastore_search\', \n                 params=query, \n                 headers=headers, \n                 verify=SSL

### Importiere die Monatsbevölkerung

Dataset auf INTEG Datenkatalog: https://data.integ.stadt-zuerich.ch/dataset/int_dwh_bev_monat_bestand_quartier_geschl_ag_herkunft_od3250

(int_dwh)

In [221]:
# Filepath
fp = r"//szh.loc/ssz/applikationen/OGD_Dropzone/INT_DWH/bev_monat_bestand_quartier_geschl_ag_herkunft_od3250/BEV325OD3250.csv"

# Read the data
imp_data2betested = pd.read_csv(
    fp, 
    sep=',',
   #usecols=[0,1,2,3,4,6],
   #names=['stationid','year','month','avgtemp','latitude','longitude','elevation','station','countryid','country'],
   #skiprows=1,
   # parse_dates=[0],
    parse_dates=True,
    date_parser=lambda x: pd.to_datetime(x, format="%Y"), 
    #index_col="date_time",
    low_memory=False    
    )

In [222]:
imp_data2betested.info()
# Find the number of null values for all columns
#data2betested.isnull().sum()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 190680 entries, 0 to 190679
Data columns (total 17 columns):
 #   Column            Non-Null Count   Dtype 
---  ------            --------------   ----- 
 0   StichtagDatJahr   190680 non-null  int64 
 1   StichtagDatMM     190680 non-null  int64 
 2   StichtagDatMonat  190680 non-null  object
 3   StichtagDat       190680 non-null  int64 
 4   SexCd             190680 non-null  int64 
 5   SexLang           190680 non-null  object
 6   AlterV20Sort      190680 non-null  int64 
 7   AlterV20Kurz      190680 non-null  object
 8   HerkunftCd        190680 non-null  int64 
 9   HerkunftLang      190680 non-null  object
 10  KreisCd           190680 non-null  int64 
 11  KreisLang         190680 non-null  object
 12  QuarCd            190680 non-null  int64 
 13  QuarLang          190680 non-null  object
 14  DatenstandCd      190680 non-null  object
 15  DatenstandLang    190680 non-null  object
 16  AnzBestWir        190680 non-null  int

Checke welche Quartiere vorkommen

In [223]:
imp_data2betested['QuarLang'].unique()

array(['Rathaus', 'Hochschulen', 'Lindenhof', 'City', 'Wollishofen',
       'Leimbach', 'Enge', 'Alt-Wiedikon', 'Friesenberg', 'Sihlfeld',
       'Werd', 'Langstrasse', 'Hard', 'Gewerbeschule', 'Escher Wyss',
       'Unterstrass', 'Oberstrass', 'Fluntern', 'Hottingen', 'Hirslanden',
       'Witikon', 'Seefeld', 'Mühlebach', 'Weinegg', 'Albisrieden',
       'Altstetten', 'Höngg', 'Wipkingen', 'Affoltern', 'Oerlikon',
       'Seebach', 'Saatlen', 'Schwamendingen-Mitte', 'Hirzenbach',
       'Unbekannt'], dtype=object)

In [224]:
# Ist da eine 1-1 Verbindung zwischen 'QuarCd', 'QuarLang'?
#------------------
#imp_data2betested[['QuarCd', 'QuarLang']].drop_duplicates()
#imp_data2betested

Folgend wird gezeigt, wie columns neu bennant, weggelassen oder neu berechnet werden können. 

In [225]:
data2betested = imp_data2betested \
    .copy() \
    .drop(columns_to_drop, axis=1) \
    .rename(columns=rename_cols) \
    .assign(
        #num_AnzBestWir_quart=lambda x: x.groupby(['QuarCd','QuarLang','StichtagDatJahr','StichtagDatMonat'])['QuarCd'].transform('sum'),
        #spots_plz=lambda x: x.groupby(['district', 'plz'])['spots'].transform(np.sum)
    )
    
data2betested

,StichtagDatJahr,StichtagDatMM,StichtagDatMonat,StichtagDat,SexCd,SexLang,AlterV20Sort,AlterV20Kurz,HerkunftCd,HerkunftLang,KreisCd,KreisLang,QuarCd,QuarLang,DatenstandCd,DatenstandLang,AnzBestWir
0,1998,1,Januar,13910,1,männlich,1,0-19,1,Schweizer/in,1,Kreis 1,11,Rathaus,V,Veröffentlicht,114
1,1998,1,Januar,13910,1,männlich,1,0-19,1,Schweizer/in,1,Kreis 1,12,Hochschulen,V,Veröffentlicht,24
2,1998,1,Januar,13910,1,männlich,1,0-19,1,Schweizer/in,1,Kreis 1,13,Lindenhof,V,Veröffentlicht,37
3,1998,1,Januar,13910,1,männlich,1,0-19,1,Schweizer/in,1,Kreis 1,14,City,V,Veröffentlicht,48
4,1998,1,Januar,13910,1,männlich,1,0-19,1,Schweizer/in,2,Kreis 2,21,Wollishofen,V,Veröffentlicht,950
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
190675,2021,5,Mai,22431,2,weiblich,5,80 u. älter,2,Ausländer/in,11,Kreis 11,119,Seebach,P,Provisorisch,89
190676,2021,5,Mai,22431,2,weiblich,5,80 u. älter,2,Ausländer/in,12,Kreis 12,121,Saatlen,P,Provisorisch,17
190677,2021,5,Mai,22431,2,weiblich,5,80 u. älter,2,Ausländer/in,12,Kreis 12,122,Schwamendingen-Mitte,P,Provisorisch,40
190678,2021,5,Mai,22431,2,weiblich,5,80 u. älter,2,Ausländer/in,12,Kreis 12,123,Hirzenbach,P,Provisorisch,60


### Kitas
Importiere die Daten «[Kennzahlen zu Kitas (Kindertagesstätten) der Stadt Zürich nach Stadtquartier, seit 2014](https://data.stadt-zuerich.ch/dataset/sd_zv_kitas_stadtquartier)»

In [226]:
# Filepath
#fp = r"//szh.loc/ssz/applikationen/OGD_Dropzone/DWH/sd_zv_kitas_stadtquartier/sd_zv_kitas_stadtquartier.csv"
url =r"https://data.stadt-zuerich.ch/dataset/sd_zv_kitas_stadtquartier/download/sd_zv_kitas_stadtquartier.csv"
# Read the data
imp_kitas_stadtquartier = pd.read_csv(
    url, 
    sep=',',
   #usecols=[0,1,2,3,4,6],
   #names=['stationid','year','month','avgtemp','latitude','longitude','elevation','station','countryid','country'],
   #skiprows=1,
   # parse_dates=[0],
    #parse_dates=True,
    #date_parser=lambda x: pd.to_datetime(x, format="%Y"), 
    #index_col="date_time",
    low_memory=False    
    )

In [227]:
imp_kitas_stadtquartier.head()
#imp_kitas_stadtquartier['Qname'].unique()
#imp_data2betested[['QNr', 'Qname']].drop_duplicates()
#imp_data2betested.shape



,QNr,Qname,Skname,Jahr,Anz_BP_SQ,Anz_Kinder_VA,Belegungsfaktor,Versorgungsquote
0,11,Rathaus,Zürichberg,2014,12.5,94,2,0
1,11,Rathaus,Zürichberg,2015,12.5,86,2,0
2,11,Rathaus,Zürichberg,2016,12.5,76,2,0
3,11,Rathaus,Zürichberg,2017,47,76,2,1
4,11,Rathaus,Zürichberg,2018,0,87,2,0


In [233]:
# Rename columns bei Bedarf
rename_cols = {
    #'Anz_BP_SQ': 'Betreuungsplaetze_in_Stadtquartieren',
    #'Anz_Kinder_VA': 'Anzahl_Kinder_Vorschulalter',
}

columns_to_drop = []
# Format data: remove redundant columns, rename columns and add new features.

kitas_stadtquartier = imp_kitas_stadtquartier \
    .copy() \
    .drop(columns_to_drop, axis=1) \
    .rename(columns=rename_cols) \
    .assign(
        #num_AnzBestWir_quart=lambda x: x.groupby(['QuarCd','QuarLang','StichtagDatJahr','StichtagDatMonat'])['QuarCd'].transform('sum'),
        #spots_plz=lambda x: x.groupby(['district', 'plz'])['spots'].transform(np.sum)
    )

kitas_stadtquartier.head(500)

,QNr,Qname,Skname,Jahr,Anz_BP_SQ,Anz_Kinder_VA,Belegungsfaktor,Versorgungsquote
0,11,Rathaus,Zürichberg,2014,12.5,94,2,0
1,11,Rathaus,Zürichberg,2015,12.5,86,2,0
2,11,Rathaus,Zürichberg,2016,12.5,76,2,0
3,11,Rathaus,Zürichberg,2017,47,76,2,1
4,11,Rathaus,Zürichberg,2018,0,87,2,0
...,...,...,...,...,...,...,...,...
233,123,Hirzenbach,Schwamendingen,2016,136.5,764,2,0
234,123,Hirzenbach,Schwamendingen,2017,189,813,2,0
235,123,Hirzenbach,Schwamendingen,2018,202,870,2,0
236,123,Hirzenbach,Schwamendingen,2019,226.5,879,2,0


## Explorative Analyse
One of the main objectives of this notebook is to do an exploratory data analysis to understand which questions this data set can answer. In addition, also determine its limitations. 

To begin, let us get the number of unique values per feature.


In [211]:
data2betested.apply(lambda x: x.unique().size, axis=0)

StichtagDatJahr       24
StichtagDatMM         12
StichtagDatMonat      12
StichtagDat          281
SexCd                  2
SexLang                2
AlterV20Sort           5
AlterV20Kurz           5
HerkunftCd             2
HerkunftLang           2
KreisCd               13
KreisLang             13
QuarCd                35
QuarLang              35
DatenstandCd           2
DatenstandLang         2
AnzBestWir          3243
dtype: int64

In [212]:
kitas_stadtquartier.apply(lambda x: x.unique().size, axis=0)
kitas_stadtquartier.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 238 entries, 0 to 237
Data columns (total 8 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   QNr               238 non-null    int64  
 1   Qname             238 non-null    object 
 2   Skname            238 non-null    object 
 3   Jahr              238 non-null    int64  
 4   Anz_BP_SQ         238 non-null    object 
 5   Anz_Kinder_VA     238 non-null    float64
 6   Belegungsfaktor   238 non-null    float64
 7   Versorgungsquote  238 non-null    float64
dtypes: float64(3), int64(2), object(3)
memory usage: 15.0+ KB


In [214]:
kitas_stadtquartier \
    .groupby(['Jahr','Skname' ]) \
    .agg(anz_betreungsplaetze_sk=('Anz_BP_SQ', 'sum'), anz_kinder_va_sk=('Anz_Kinder_VA', 'sum')) \
    .query('Jahr > 2016') \
    .sort_values('anz_kinder_va_sk', ascending=False) \
    .head(500)

,,anz_betreungsplaetze_sk,anz_kinder_va_sk
Jahr,Skname,,
2018,Glattal,395720.5590.5,4541
2017,Glattal,364.5669.5590,4524
2019,Glattal,418.5737.5632,4381
2020,Glattal,419756.5711,4294
2019,Waidberg,634.1331.9506501.5,4258
2020,Waidberg,666.6325.3524.5485,4252
2018,Waidberg,581.6317.9448474.5,4250
2017,Waidberg,554.6305.4398439,4123
2018,Uto,45125262.5822153,4117
